# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [39]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need

from pyspark.sql.functions import desc, udf, col
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

# 2) instantiate a Spark session 

spark = SparkSession \
    .builder \
    .appName('Wrangling Data') \
    .getOrCreate()

# 3) read in the data set located at the path "data/sparkify_log_small.json"

path = 'data/sparkify_log_small.json'
user_log = spark.read.json(path)

# 4) write code to answer the quiz questions 

user_log.take(5)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046'),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000'),
 Row(artist='Cobra Starship Featuring Leighton Meester', auth='Logged In', firstNa

In [31]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [32]:
# TODO: write your code to answer question 1

all_pages = user_log.select('page').dropDuplicates()

blank_user_id_pages = user_log.select('page').where(user_log.userId == '').dropDuplicates()

blank_user_id_not_visited_pages = set(all_pages.collect()) - set(blank_user_id_pages.collect())

print(blank_user_id_not_visited_pages)

{Row(page='Upgrade'), Row(page='Logout'), Row(page='Settings'), Row(page='Submit Downgrade'), Row(page='NextSong'), Row(page='Downgrade'), Row(page='Save Settings'), Row(page='Submit Upgrade'), Row(page='Error')}


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [33]:
# TODO: use this space to explore the behavior of the user with an empty string

Meaby dummy user used for testing the system

# Question 3

How many female users do we have in the data set?

In [34]:
# TODO: write your code to answer question 3

user_log.select('gender').dropDuplicates().collect()

[Row(gender='F'), Row(gender=None), Row(gender='M')]

In [35]:
user_log.select('userid', 'gender') \
    .where(user_log.gender == 'F') \
    .dropDuplicates() \
    .count()

462

# Question 4

How many songs were played from the most played artist?

In [43]:
# TODO: write your code to answer question 4

user_log.filter(user_log.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .sort(desc('count(Artist)')) \
    .show(1)

+--------+-------------+
|  Artist|count(Artist)|
+--------+-------------+
|Coldplay|           83|
+--------+-------------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [40]:
# TODO: write your code to answer question 5

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cumulative_sum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cumulative_sum.filter((cumulative_sum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

